# Deploy your model on a SageMaker Endpoint and visualize predictions

## Setup environment

In [2]:
import cv2
import sagemaker
import numpy as np
import matplotlib.pyplot as plt
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.tensorflow import TensorFlowModel
import visualization_utils as viz_utils


import sagemaker, boto3, os

bucket = sagemaker.Session().default_bucket()
print(f'def bucket: {str(bucket)}')

artifact_version = 1
prefix = "form-fields-det-model-artifact"
model_artifact_name = f'form-field-det-v{artifact_version}.tar.gz'

role = sagemaker.get_execution_role()

model_artefact = f's3://{str(bucket)}/{prefix}/{model_artifact_name}'

def bucket: sagemaker-us-east-1-753177091730


In [3]:
pip install -U sagemaker

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Create endpoint

In [6]:
container_name = '753177091730.dkr.ecr.us-east-1.amazonaws.com/sagemaker-form-field-detection:20210809131501'
image_name = 'sagemaker-form-field-detection'
model = TensorFlowModel(
    name=name_from_base(image_name),
    model_data=model_artefact,
    role=role,
    framework_version='2.4'
)

In [7]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------!

In [11]:
predictor

## Predict

In [9]:
def image_file_to_tensor(path):
    cv_img = cv2.imread(path,1).astype('uint8')
    cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    return cv_img

In [14]:
def image_file_to_tensor_2(path):
    """Reads an image file and coverts it to a tensor (ndarray).
    Resizing of input is done (1024x1024 for the efficientdet model).
    Args:
        path (str): The file name or path to the image file.
    """

    image = cv2.imread(path)
    image = cv2.resize(image, dsize=(1024, 1024), interpolation=cv2.INTER_CUBIC)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.asarray(image)
#     image = cv2.normalize(image.astype("float"), None, 0, 1, cv2.NORM_MINMAX)
    image = np.expand_dims(image, axis=0)
    return image

In [15]:
img = image_file_to_tensor_2('test_images/test-1.png')

In [16]:
detections = predictor.predict(img)

In [21]:
detections = detections['predictions'][0]

In [22]:
detection_boxes = detections['detection_boxes']
detection_classes = detections['detection_classes']
detection_scores = detections['detection_scores']

In [ ]:
detection_scores

In [ ]:
#### INCORRECT!

img = image_file_to_tensor('test_images/test-1.png')

input = {
  'instances': [img.tolist()]
}

detections = predictor.predict(input)['predictions'][0]

img_with_detections = img.copy()
category_index = {1:{'id': 1, 'name': 'bee'}}
detection_boxes = np.array(detections['detection_boxes'])
detection_classes = [int(x) for x in detections['detection_classes']]
detection_scores = detections['detection_scores']

In [ ]:
%matplotlib inline

viz_utils.visualize_boxes_and_labels_on_image_array(
    img_with_detections,
    detection_boxes,
    detection_classes,
    detection_scores,
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=100,
    min_score_thresh=0.60,
    agnostic_mode=False)

plt.figure(figsize=(10,10))
plt.imshow(img_with_detections)
plt.show()

## Delete endpoint

In [14]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
